In [ ]:
# import modules
import pandas as pd # for dataframes
import seaborn as sns # for plotting graphs
import matplotlib.pyplot as plt # for plotting graphs

from datetime import datetime as dt

In [ ]:
# URL for the data file and read it into the data frame
url = 'https://raw.githubusercontent.com/bhascarp/Data_Files/main/rfmdata.csv'
rfm_df = pd.read_csv(url)

In [ ]:
rfm_df.head()

,Row ID,Order ID,Order Date,Customer ID,Sales
0,1,CA-2016-152156,11/8/2019,CG-12520,261.96
1,2,CA-2016-152156,11/8/2019,CG-12520,731.94
2,3,CA-2016-138688,6/12/2019,DV-13045,14.62
3,4,US-2015-108966,10/11/2018,SO-20335,957.58
4,5,US-2015-108966,10/11/2018,SO-20335,22.37


In [ ]:
rfm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8098 entries, 0 to 8097
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Row ID       8098 non-null   int64  
 1   Order ID     8098 non-null   object 
 2   Order Date   8098 non-null   object 
 3   Customer ID  8098 non-null   object 
 4   Sales        8098 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 316.5+ KB


In [ ]:
rfm_df['Order Date'] = pd.to_datetime(rfm_df['Order Date'])


since our analysis is based on customers, we will remove if any missing values

In [ ]:
rfm_df= rfm_df[pd.notnull(rfm_df['Customer ID'])]


In [ ]:
rfm_df['Order Date'].min(),rfm_df['Order Date'].max()


(Timestamp('2017-01-03 00:00:00'), Timestamp('2020-07-31 00:00:00'))

In [ ]:
import datetime as dt

current = dt.datetime(2020,8,1)

rfm_df['Order Date'] = pd.to_datetime(rfm_df['Order Date'])


RFM Customer Segmentation


In [ ]:
rfmTable = rfm_df.groupby('Customer ID').agg({'Order Date': lambda x: (current - x.max()).days, 'Order ID': lambda x: len(x), 'Sales': lambda x: x.sum()})

rfmTable['Order Date'] = rfmTable['Order Date'].astype(int)

rfmTable.rename(columns={'Order Date': 'recency', 
                         'Order ID': 'frequency', 
                         'Sales': 'monetary_value'}, inplace=True)

Calculate RFM metrics for each customer



In [ ]:
rfmTable.head()

,recency,frequency,monetary_value
Customer ID,,,
AA-10315,33,11,5563.56
AA-10375,261,10,849.66
AA-10480,108,12,1790.51
AA-10645,332,17,5073.98
AB-10015,265,6,886.15


Let's check the details of first customer

In [ ]:
first_customer = rfm_df[rfm_df['Customer ID'] == 'CG-12520']
first_customer

,Row ID,Order ID,Order Date,Customer ID,Sales
0,1,CA-2016-152156,2019-11-08,CG-12520,261.96
1,2,CA-2016-152156,2019-11-08,CG-12520,731.94
4439,5492,CA-2017-164098,2020-01-26,CG-12520,18.16
5565,6878,US-2015-123918,2018-10-15,CG-12520,131.38
5566,6879,US-2015-123918,2018-10-15,CG-12520,5.34


Split the metrics

In [ ]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

Create a segmented RFM table



In [ ]:
segmented_rfm = rfmTable

The lowest recency, highest frequency and highest monetary amounts are our best customers.

In [ ]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

Add segment numbers to the newly created segmented RFM table



In [ ]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
Customer ID,,,,,,
AA-10315,33,11,5563.56,1,2,1
AA-10375,261,10,849.66,3,2,3
AA-10480,108,12,1790.51,2,2,2
AA-10645,332,17,5073.98,4,1,1
AB-10015,265,6,886.15,3,4,3


Add a new column to combine RFM score: 111 is the highest score as we determined earlier.

In [ ]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
Customer ID,,,,,,,
AA-10315,33,11,5563.56,1,2,1,121
AA-10375,261,10,849.66,3,2,3,323
AA-10480,108,12,1790.51,2,2,2,222
AA-10645,332,17,5073.98,4,1,1,411
AB-10015,265,6,886.15,3,4,3,343


Top 10 of best customers

In [ ]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
Customer ID,,,,,,,
SM-20320,23,14,25035.10,1,1,1,111
RB-19360,7,16,14986.78,1,1,1,111
GT-14710,35,25,11721.78,1,1,1,111
CJ-12010,38,20,11164.97,1,1,1,111
ME-17320,50,19,10393.23,1,1,1,111
TB-21400,51,17,9133.99,1,1,1,111
LA-16780,3,26,8673.23,1,1,1,111
JE-15715,47,20,8637.92,1,1,1,111
ZC-21910,43,28,7996.01,1,1,1,111
